### About me:

- Name: Khaled
- Surname: Hechmi
- Email: k.hechmi@campus.unimib.it
- Student ID: 793085

## Import libraries

In [ ]:
import pandas as pd
import numpy as np
from datetime import date
import requests
import json

## 1. Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.
The dataset of interest is loans_lenders.csv, therefore let's download it:

In [ ]:
loans_lenders_df = pd.read_csv("additional-kiva-snapshot/loans_lenders.csv")

### Basic data exploration and statistics

In [ ]:
loans_lenders_df.head()

In [ ]:
loans_lenders_df.tail()

How many records are there?

In [ ]:
loans_lenders_df.shape

How many NA values are in this dataframe?

In [ ]:
loans_lenders_df.isna().sum()

Are there duplicated loan_id values?

In [ ]:
len(loans_lenders_df['loan_id'].unique())

That's good, each id is unique. I can now focus on how to normalize the table: the first thing to do will be to split the strings in the lenders column, so that we can have a list of lenders

In [ ]:
loans_lenders_df['lenders'] = loans_lenders_df['lenders'].apply(lambda x : x.split(','))

In [ ]:
loans_lenders_df = loans_lenders_df.explode('lenders').reset_index(drop=True)

Let's see if everything work as expected:

In [ ]:
loans_lenders_df.head()

In [ ]:
loans_lenders_df.tail()

Just for curiosity let's have a look at a random row:

In [ ]:
loans_lenders_df.iloc[45]

In [ ]:
loans_lenders_df.shape

## 2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

The first step is to load the loans dataset

In [ ]:
loans_df = pd.read_csv("additional-kiva-snapshot/loans.csv")

In [ ]:
loans_df.columns

In [ ]:
loans_df.head()

In [ ]:
loans_df.tail()

In [ ]:
loans_df.describe()

There are many columns! At the moment only disburse_time and planned_expiration_time seems relevant, together with loan_id, therefore it is better to filter the df!

In [ ]:
columns_of_interest = ['loan_id', 'disburse_time','planned_expiration_time']

In [ ]:
loans_filtered = loans_df[columns_of_interest]

In [ ]:
loans_filtered.head()

In [ ]:
loans_filtered.tail()

In [ ]:
loans_filtered.shape

In [ ]:
len(loans_filtered['loan_id'].unique())

No duplicate rows, this is good. Let's now have a look at the two variables:

In [ ]:
loans_filtered.disburse_time.describe()

In [ ]:
loans_filtered.planned_expiration_time.describe()

They are seen as a generic object from Pandas even though they are dates.
How many NAs are there?

In [ ]:
loans_filtered.disburse_time.isna().sum()

In [ ]:
loans_filtered.planned_expiration_time.isna().sum()

Now let's convert these two variable in appropriate datetime objects

In [ ]:
loans_filtered.disburse_time = pd.to_datetime(loans_filtered.disburse_time)

In [ ]:
loans_filtered.planned_expiration_time = pd.to_datetime(loans_filtered.planned_expiration_time)

Now computing the difference between these two columns is easy:

In [ ]:
loans_filtered['diff_expiration_disburse'] = loans_filtered.planned_expiration_time - loans_filtered.disburse_time

In [ ]:
loans_filtered.head()

In [ ]:
loans_filtered.describe()

In [ ]:
loans_filtered.diff_expiration_disburse.isna().sum()

Is the number plausible? It should be less or equal than the number of NAs in the two columns:

In [ ]:
loans_filtered.disburse_time.isna().sum() + loans_filtered.planned_expiration_time.isna().sum()

Apparently yes! It means that in 2 scenarios both planned_expiration_time and disburse_time where NAs. Let's see where:

In [ ]:
loans_filtered[loans_filtered[['disburse_time', 'planned_expiration_time']].isna().all(axis=1)]

## 3. Find the lenders that have funded at least twice.

It is possible to reuse the dataframe create at Q1. Grouping data by the lenders name and counting the frequencies will be enough for answering the question.

In [ ]:
funding_freq = loans_lenders_df.groupby('lenders').lenders.count()
funding_freq

In [ ]:
funding_freq = funding_freq.to_frame()

In [ ]:
funding_freq

We are interested only on lenders who have funded at least twice, therefore let's filter the rows accordingly

In [ ]:
funding_freq[funding_freq.lenders >= 2]

## 4. For each country, compute how many loans have involved that country as borrowers.

In [ ]:
loans_df.columns

Let's filter, once again, the loans dataset keeping only the columns of interest:
- loan_id
- country_code
- country_name

In [ ]:
loans_filtered['country_code'] = loans_df['country_code']
loans_filtered['country_name'] = loans_df['country_name']

In [ ]:
loans_filtered.head()

We have add the information related to each loan country. Now it is really easy to compute the answer:

In [ ]:
loans_country = loans_filtered.country_name.value_counts()
loans_country

##  5. For each country, compute the overall amount of money borrowed.

In [ ]:
loans_df.columns

In the loans dataset there is a column named "currency", that indicates the currency of a given amount. Therefore it is important to convert the 'loan_amount' to a reference currency (USD)

In [ ]:
loans_filtered['loan_amount'] = loans_df['loan_amount']
loans_filtered['currency'] = loans_df['currency']
loans_filtered.head()

Which are the currencies in our dataset?

In [ ]:
loans_filtered['currency'].unique()

For converting these currencies I will use the free [Currency Converter API](https://www.currencyconverterapi.com/). Please keep in mind that this approach, even though it converts all the loan_amount to the same currency, is not ideal because:
- It does not take into consideration the exchange rate in the time period where the loans were taken
- It does not take into consideration inflation rates across the different years

In [ ]:
%%time
url_currency = "https://free.currconv.com/api/v7/convert?q={}&compact=ultra&apiKey=34929da3012332915e3e"
results = {}
for currency in loans_filtered['currency'].unique():
    conversion_query="{}_USD".format(currency)
    current_url = url_currency.format(conversion_query)
    result = requests.get(current_url).content
    try:
        results[currency] = json.loads(result)[conversion_query]
    except:
        print("{} value not found".format(conversion_query))
        results[currency] = None
    #results[currency] = requests.get(url_currency.format(currency)).content

    

- [SSP](https://www.google.com/search?hl=en&ei=WqxOXuCVD8f76QTez6UI&q=ssp+currency&oq=ssp+currency&gs_l=psy-ab.3..0l3j0i22i30l3.3353.4669..4907...0.2..0.120.929.4j5......0....1..gws-wiz.......0i71j0i67.k8hD5JFknpI&ved=0ahUKEwjgqpCrv-DnAhXHfZoKHd5nCQEQ4dUDCAs&uact=5) is the South Sudanese Pound and it replaces the Sudanese Pound [SDG](https://www.google.com/search?hl=en&ei=YKxOXqjVDoyCk74PmoaLsAc&q=sudanese+pound+currency&oq=sudanese+pound+currency&gs_l=psy-ab.3..0i22i30l5.50934.54867..55137...0.4..0.150.2411.9j14......0....1..gws-wiz.......0i71j0i67j0j0i67i70i251j0i70i258j0i8i13i30.hIs6AnVdQ94&ved=0ahUKEwiohf6tv-DnAhUMwcQBHRrDAnYQ4dUDCAs&uact=5)
- ZWD is not in the list. Today (22/02) the exchange rate according to XE.com is 1 ZWD = 0.00276319 USD

In [ ]:
results['ZWD'] = 0.00276319
results['SSP'] = results['SDG']

Let's have a look at the exchange rates:

In [ ]:
results

Now it is possible to convert the different loans amount into USD:

In [ ]:
%%time
for currency in results.keys():
    currency_index = loans_filtered.index[loans_filtered['currency'] == currency]
    loans_filtered.loc[currency_index, 'loan_amount'] = loans_filtered.loc[currency_index, 'loan_amount'] * results[currency]
    

In [ ]:
loans_filtered.head()

Now for answering the starting question it is sufficient to group data according to their country and sum the loan_amount values

In [ ]:
overall_money_borrowed = loans_filtered.groupby('country_name')['loan_amount'].agg(Money_borrowed='sum')
overall_money_borrowed

## 6. Like the previous point, but expressed as a percentage of the overall amount lent.

First step = finding the overall amount lent

In [ ]:
overall_amount_lent = loans_filtered.loan_amount.sum()
overall_amount_lent

Now we can reuse the previously computed dataset and transform its values into percentages

In [ ]:
money_borrowed_perc = (overall_money_borrowed/overall_amount_lent) * 100
money_borrowed_perc

If everything is correct the column Money_borrowed_perc should sum to 100. Let's find out:

In [ ]:
money_borrowed_perc.Money_borrowed.sum()

Thats good! The result is not exactly 100 because of decimal numbers approximation

## 7. Like the three previous points, but split for each year (with respect to disburse time).

In [ ]:
loans_filtered.head()

In [ ]:
loans_filtered['disburse_time'].describe()

In [ ]:
type(loans_filtered['disburse_time'])

In [ ]:
loans_filtered['disburse_year'] = loans_filtered['disburse_time'].dt.year

### For each country, compute how many loans have involved that country as borrowers.

In [ ]:
loans_country_year = loans_filtered.groupby(['country_name', 'disburse_year'])['loan_id'].agg(Loans_freq='count')
loans_country_year

For being sure to have done everything correctly let's compare the results obtained at this stage for Afghanistan and the ones obtained by the same countryat task 4 

In [ ]:
loans_country_year.loc['Afghanistan']

In [ ]:
loans_country_year.loc['Afghanistan'].sum()

In [ ]:
loans_country['Afghanistan']

The results are equal, therefore the procedure is ok

### For each country, compute the overall amount of money borrowed.

In [ ]:
overall_money_borrowed_year = loans_filtered.groupby(
    ['country_name', 'disburse_year'])['loan_amount'].agg(Money_borrowed='sum')
overall_money_borrowed_year

Let's perform the same check:

In [ ]:
overall_money_borrowed_year.loc['Afghanistan']

In [ ]:
overall_money_borrowed_year.loc['Afghanistan'].sum()

In [ ]:
overall_money_borrowed.loc['Afghanistan'].sum()

Everything looks fine

### Like the previous point, but expressed as a percentage of the overall amount lent.

In [ ]:
money_borrowed_year_perc = (overall_money_borrowed_year/overall_amount_lent) * 100
money_borrowed_year_perc

Once again let's check the results:

In [ ]:
money_borrowed_year_perc.Money_borrowed.sum()

Considering the numbers of countries and the number of year losing 2.9% due to decimal approximation seems reasonable. Let's now check Afghanistan results:

In [ ]:
money_borrowed_year_perc.loc['Afghanistan'].sum()

In [ ]:
100 * (overall_money_borrowed_year.loc['Afghanistan'].sum()/overall_amount_lent)

The results seems good also in this case.

## 8. For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [ ]:
loans_df.head()

In [ ]:
loans_df.shape

In [ ]:
len(loans_df.loan_id.unique())

In [ ]:
loans_lenders_df

The first step will be to compute the number of people involved in each loan

In [ ]:
loan_id_number_of_lenders= loans_lenders_df.groupby('loan_id')['lenders'].agg(number_of_lenders="count")
loan_id_number_of_lenders

Let's see if the result is correct by manually checking some loans:

In [ ]:
loans_lenders_df[loans_lenders_df.loan_id == 84]

In [ ]:
loans_lenders_df[loans_lenders_df.loan_id == 85]

In [ ]:
loans_lenders_df[loans_lenders_df.loan_id == 1444065]

Let's now create a table where only the loan_id and the amount of money lent is present:

In [ ]:
loan_id_amount = loans_filtered.filter(['loan_id', 'loan_amount'])
loan_id_amount.head()

For finding the answer it is necessary to have a dataframe where loan_id, lender names, number of lenders and loan_amount are there. Therefore the following operation needs to be done: loans_lenders_df JOIN number_of_lenders JOIN loan_id_amount

In [ ]:
loans_lenders_join_number_of_lenders = loans_lenders_df
loans_lenders_join_number_of_lenders= loans_lenders_join_number_of_lenders.join(loan_id_number_of_lenders, on="loan_id")

In [ ]:
loans_lenders_join_number_of_lenders_join_loan_id_amount = loans_lenders_join_number_of_lenders.join(loan_id_amount, on='loan_id', lsuffix='',rsuffix= '_right')

In [ ]:
loans_lenders_join_number_of_lenders_join_loan_id_amount.head()

In [ ]:
del loans_lenders_join_number_of_lenders_join_loan_id_amount['loan_id_right']
loans_lenders_join_number_of_lenders_join_loan_id_amount.head()

Let's now create a new column that contains the contribution of each lender to the final loan:

In [ ]:
loans_lenders_join_number_of_lenders_join_loan_id_amount['loan_amount_per_lender'] = loans_lenders_join_number_of_lenders_join_loan_id_amount['loan_amount'] / loans_lenders_join_number_of_lenders_join_loan_id_amount['number_of_lenders']
loans_lenders_join_number_of_lenders_join_loan_id_amount.head()

Now it is possible to find the final answer:

In [ ]:
money_lent_per_lenders = loans_lenders_join_number_of_lenders_join_loan_id_amount.groupby('lenders')['loan_amount_per_lender'].agg(money_lent='sum')
money_lent_per_lenders

## 9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed.
Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

Let's load the lenders dataset:

In [ ]:
lenders  = pd.read_csv("additional-kiva-snapshot/lenders.csv")
lenders.head()

In [ ]:
loans_lenders_df.head()

### Money lent

In order to find the lent amoun money for each country we need to have in a single df the loan_ids, the lenders names and their country:

In [ ]:
lenders_loan_id = pd.merge(loans_lenders_df, lenders[['permanent_name', 'country_code']],left_on="lenders", right_on="permanent_name")

In [ ]:
lenders_loan_id.head()

Now it is needed to check if there are any NAs in the country column: for fill them NA it will be taken into consideration the data distribution. Before doing that, in order to avoid distorting data, we will remove all the duplicated rows: as we have seen in the previous tasks there are a huge number of lenders who have funded more than one loan

In [ ]:
lenders_country = lenders_loan_id[['lenders', 'country_code']].drop_duplicates()
lenders_country.head()

How many NAs are there?

In [ ]:
missing_country_code = lenders_country.country_code.isnull()
sum(missing_country_code)

In [ ]:
unique_countries_distribution = lenders_country.country_code.value_counts(normalize=True)
unique_countries_distribution

Now it is possible to replace the values:

In [ ]:
len(missing_country_code.index)

In [ ]:
len(lenders_country[missing_country_code])

In [ ]:
lenders_country.loc[missing_country_code,'country_code'] = np.random.choice(unique_countries_distribution.index,
                                                                            size=len(lenders_country[missing_country_code]),
                                                                            p=unique_countries_distribution.values)


In [ ]:
lenders_country.head()

How many NAs?

In [ ]:
missing_country_code = lenders_country.country_code.isnull()
sum(missing_country_code)

We can now add to the dataset the overall amount of money that each lender has lent, using the answer at the previous question:

In [ ]:
money_lent_per_lender_country = pd.merge(lenders_country, money_lent_per_lenders, left_on="lenders", right_index=True)
money_lent_per_lender_country

Now computing the amount of money lent by each country requires just a group by country and a sum of the money_lent values

In [ ]:
money_lent_by_country = money_lent_per_lender_country.groupby('country_code')['money_lent'].agg(overall_money_lent="sum")
money_lent_by_country

### Money borrowed
The answer to this question was found at Q5:

In [ ]:
overall_money_borrowed

The only problem is that in one dataset we have the full name, in the other its 2-letters version. Let's add in the borrowed dataset this new column:

In [ ]:
country_code_name = loans_df[['country_code', 'country_name']].drop_duplicates()
country_code_name.head()

In [ ]:
country_code_name.shape

Are there any NA?

In [ ]:
sum(country_code_name.country_code.isna())

In [ ]:
sum(country_code_name.country_name.isna())

In [ ]:
country_code_name[country_code_name.country_code.isna()]

Let's replace this value with NA, that, according to the ISO standard, is the correct code for Nambibia. Source: https://en.wikipedia.org/wiki/ISO_3166-2:NA

In [ ]:
country_code_name.loc[country_code_name.country_code.isna(),'country_code'] = "NA"

In [ ]:
country_code_name[country_code_name.country_name == 'Namibia']

Now let's attach the country_code to the borrowers dataset

In [ ]:
money_borrowed_by_country = pd.merge(country_code_name, overall_money_borrowed, left_on="country_name", right_index=True)
money_borrowed_by_country.head()

Let's now merge the borrowers and lenders:

In [ ]:
money_lent_by_country.head()

In [ ]:
lent_borrowed_by_country = pd.merge(money_lent_by_country, money_borrowed_by_country, left_index=True, right_on='country_code')
lent_borrowed_by_country

We can now compute the difference between money lent and money borrowed:

In [ ]:
lent_borrowed_by_country['difference_lent_borr'] = lent_borrowed_by_country['overall_money_lent'] - lent_borrowed_by_country["Money_borrowed"]
lent_borrowed_by_country


## 10. Which country has the highest ratio between the difference computed at the previous point and the population?

The population infos are in the dataset country_stats.csv, therefore let's load it:

In [ ]:
country_stats = pd.read_csv("additional-kiva-snapshot/country_stats.csv")
country_stats.head()

In [ ]:
country_pop = country_stats[['country_name', 'population']]
country_pop.head()

Now we can join the two datasets in order to have the population for each country

In [ ]:
country_pop_money_stats = pd.merge(lent_borrowed_by_country, country_pop, left_on="country_name", right_on="country_name")
country_pop_money_stats.head()

Computing the ratio now is easy: a ratio of the two columns is enough:

In [ ]:
country_pop_money_stats['ratio_diff_money_pop']= country_pop_money_stats['difference_lent_borr']/country_pop_money_stats['population']
country_pop_money_stats.head()

In [ ]:
country_pop_money_stats.sort_values(by=['ratio_diff_money_pop'], ascending=False)

The country with the highest ratio is Canada, with a value of 0.094904

## 11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [ ]:
country_stats.describe()

Population below poverty line appears to be a percentage. Therefore the first step is to the compute the population NOT below the poverty line.

In [ ]:
country_stats['pop_above_poverty'] = (country_stats['population']/100) * (100 -country_stats['population_below_poverty_line'])
country_stats.head()

Now I can compute the desired ratio similar to the previous point.

In [ ]:
country_poverty = country_stats[['country_name', 'pop_above_poverty']]
country_poverty.head()

In [ ]:
country_poverty_money_stats = pd.merge(lent_borrowed_by_country, country_poverty, left_on="country_name", right_on="country_name")
country_poverty_money_stats.head()

In [ ]:
country_poverty_money_stats['ratio_diff_money_pop_above_pov']= country_poverty_money_stats['difference_lent_borr']/country_poverty_money_stats['pop_above_poverty']
country_poverty_money_stats.head()

In [ ]:
country_poverty_money_stats.sort_values(by=['ratio_diff_money_pop_above_pov'], ascending=False)

Also in this case Canada has the highest ratio (0.10)

## 12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year.
For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

Should we consider both the disburse time and the planned expiration time? Let's see:

In [ ]:
date(2018, 1, 30) - date(2016, 12, 1)

The answer is yes, because in the example the total amount of days is 426: it is important, therefore, to add 1 every time the timedelta between two dates is computed in order to avoid any distortion

For Q2 I computed the difference between the two dates: it will be useful to use this info for filtering some rows:

In [ ]:
loans_filtered.shape

In [ ]:
loans_filtered.head()

In [ ]:
loans_filtered = loans_filtered[loans_filtered.diff_expiration_disburse.astype('timedelta64[D]')>0]
loans_filtered.reset_index(inplace=True)

In [ ]:
loans_filtered.diff_expiration_disburse.describe()

In [ ]:
loans_filtered.disburse_time.describe()

In [ ]:
loans_filtered.planned_expiration_time.describe()

The range of useful dates goes from 2011 to 2018 in theory.
Let's create a dictionary where each key is a year and each value is the total amount of money for loans in that year

In [ ]:
loans_amount_per_year = {}
unique_disburse_years = range(2011, 2019)
for year in unique_disburse_years:
    loans_amount_per_year[year] = 0

In [ ]:
def compute_loans_days_for_each_year(start_date, end_date, loans_amount, loan_amount):
    delta = (end_date - start_date).days
    if delta < 0: # anomaly in data
        return None
    loan_per_day = loan_amount/(delta+1)
    if start_date.year == end_date.year:
        loans_amount[start_date.year] = loans_amount[start_date.year] + (delta + 1) * loan_per_day
    elif end_date.year - start_date.year == 1:
        days_of_loans_first_year = (date(start_date.year, 12, 31) - start_date).days + 1# days_of_loans in the first years
        loans_amount[start_date.year] = loans_amount[start_date.year] + days_of_loans_first_year * loan_per_day
        loans_amount[end_date.year] = loans_amount[end_date.year] + loan_per_day * (1 +((end_date - start_date).days - days_of_loans_first_year))
    elif end_date.year - start_date.year > 1:
        days_of_loans_first_year = (date(start_date.year, 12, 31) - start_date).days + 1
        loans_amount[start_date.year] = loans_amount[start_date.year] + days_of_loans_first_year * loan_per_day
        for year in range(start_date.year + 1, end_date.year):
            loans_amount[year] = loans_amount[year] + loan_per_day * (1 + (date(year, 12, 31) - date(year, 1, 1)).days)
        loans_amount[end_date.year] = loans_amount[end_date.year] + loan_per_day * (1 + (end_date - date(end_date.year, 1, 1)).days)

Let's now test the function in the various scenarios. The first one is when the planned_expiration_time is before the disburse time: we expect to not increase any value in the dictionary because it is a clear anomaly

In [ ]:
compute_loans_days_for_each_year(date(2012,1,1), date(2011,1,30),loans_amount_per_year, 1125)

In [ ]:
loans_amount_per_year

The second scenario is when both dates are in the same year: in this case the loan amount is 30, therefore the 2012 key should have a value of 30

In [ ]:
compute_loans_days_for_each_year(date(2012,1,1), date(2012,1,30), loans_amount_per_year, 30)

In [ ]:
loans_amount_per_year

Let's now try the scenario where the two dates differ by one year.

Expected result:
- 2012 : 30 (previous run) + 732
- 2013: 2

All the other years set to 0

In [ ]:
compute_loans_days_for_each_year(date(2012,1,1), date(2013,1,1), loans_amount_per_year, 734)

In [ ]:
loans_amount_per_year

It looks good: 2012 is bisestile and according to the specification both starting and ending dates have to be considered.

Let's now test the last scenario:

In [ ]:
(date(2015,1,1) - date(2013,4,30)).days

In [ ]:
compute_loans_days_for_each_year(date(2013,4,30), date(2015,1,1), loans_amount_per_year, 612*3)

In [ ]:
loans_amount_per_year

In [ ]:
date(2013,12,31) -date(2013,4,30)

So in 2013 there are 245 + 1 days, therefore the expected value for 2013 is:

In [ ]:
246 * 3 + 2 # 2 is the value at the previous iteration

For 2014 the expected value is:

In [ ]:
365 * 3

And for 2015 the expected is 3 because only the 1st January is considered: the procedure seems correct! We can now apply it on the whole dataset.

Let's reset the results dictionary:

In [ ]:
loans_amount_per_year = {}
unique_disburse_years = range(2011, 2019)
for year in unique_disburse_years:
    loans_amount_per_year[year] = 0

In [ ]:
%%time
for i in range(0,loans_filtered.shape[0]):
    compute_loans_days_for_each_year(loans_filtered.loc[i, 'disburse_time'].date(),
                                     loans_filtered.loc[i, 'planned_expiration_time'].date(),
                                     loans_amount_per_year,
                                     loans_filtered.loc[i, 'loan_amount'])

In [ ]:
loans_amount_per_year